In [1]:
import chess.engine
import chess 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import chess.pgn 
from stockfish import Stockfish
import pandas as pd
import time
import numpy as np
import chess
import chess.pgn
import time
import multiprocessing as mp
import sys
from stockfish import Stockfish
import numpy as np
import json

In [2]:
from maia_functions import *

In [3]:
def extract_features(game):
    # Read the next game
    features = defaultdict(int)
    stockfish=Stockfish("../stockfish/stockfish-windows-x86-64-avx2.exe")
    stockfish.set_depth(15)#How deep the AI looks
    features = defaultdict(int)
    features["moves"] = [str(move) for move in game.mainline_moves()]
    features["WhiteElo"] = game.headers.get('WhiteElo', '')
    features["BlackElo"] = game.headers.get('BlackElo', '')
    features["white"] = game.headers.get('White', '')
    features["black"] = game.headers.get('Black', '')
    try:
        mm = maia_cpl(features, stockfish)
        features["maia_cpl_w"] = mm["cplw"]
        features["maia_cpl_b"] = mm["cplb"]
    except:
        return features
    return features

In [4]:
pgn_file_path = "../data/landing/FIDE_openings.pgn"

games = []
pgn_file = open(pgn_file_path)
game = chess.pgn.read_game(pgn_file)

start = 0
limit = 10000
count = 0
while game is not None:
    #features = extract_features(game)
    games.append(game)

    count += 1
    if count == limit:
        break
    game = chess.pgn.read_game(pgn_file)

pgn_file.close()

In [5]:
chunk_size = 1000  # Process 1000 games at a time
num_chunks = len(games) // chunk_size + (1 if len(games) % chunk_size > 0 else 0)
for i in range(0, num_chunks):
    result = []

    start_index = i * chunk_size
    end_index = min(start_index + chunk_size, len(games))
    games_chunk = games[start_index:end_index]

    for game in games_chunk:
        features = extract_features(game)
        result.append(features)

    ndjson_file_path = f'../data/raw/fide_maia/fide_{start_index}.ndjson'
    print(f"saved to {ndjson_file_path}")
    print("--------------------------------------------------------")
    with open(ndjson_file_path, 'w') as f:
        for item in result:
            json.dump(item, f)
            f.write('\n') 
      

<UciProtocol (pid=6436)>: stderr >>        _
<UciProtocol (pid=6436)>: stderr >> |   _ | |
<UciProtocol (pid=6436)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=6436)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=8788)>: stderr >>        _
<UciProtocol (pid=8788)>: stderr >> |   _ | |
<UciProtocol (pid=8788)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=8788)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<UciProtocol (pid=6436)>: stderr >> Loading weights file from: lc0_windows\models\maia-1400.pb.gz
<UciProtocol (pid=6436)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=6436)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=6436)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=6436)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=6436)>: stderr >> GPU: NVIDI

--weights=lc0_windows\models\maia-1400.pb.gz
--weights=lc0_windows\models\maia-1500.pb.gz


<UciProtocol (pid=8788)>: stderr >> Loading weights file from: lc0_windows\models\maia-1500.pb.gz
<UciProtocol (pid=8788)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=8788)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=8788)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=8788)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=8788)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=8788)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=8788)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=8788)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=8788)>: stderr >> L2 cache capacity: 4194304
<UciProtocol (pid=1260)>: stderr >>        _
<UciProtocol (pid=1260)>: stderr >> |   _ | |
<UciProtocol (pid=1260)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=1260)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<UciProtocol 

--weights=lc0_windows\models\maia-1900.pb.gz
--weights=lc0_windows\models\maia-1900.pb.gz


<UciProtocol (pid=24796)>: stderr >> Loading weights file from: lc0_windows\models\maia-1900.pb.gz
<UciProtocol (pid=24796)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=24796)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=24796)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=24796)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=24796)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=24796)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=24796)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=24796)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=24796)>: stderr >> L2 cache capacity: 4194304
<UciProtocol (pid=10252)>: stderr >>        _
<UciProtocol (pid=10252)>: stderr >> |   _ | |
<UciProtocol (pid=10252)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=10252)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config

...

<UciProtocol (pid=18676)>: stderr >> Loading weights file from: lc0_windows\models\maia-1700.pb.gz
<UciProtocol (pid=18676)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=18676)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=18676)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=18676)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=18676)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=18676)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=18676)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=18676)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=18676)>: stderr >> L2 cache capacity: 4194304
<UciProtocol (pid=8128)>: stderr >>        _
<UciProtocol (pid=8128)>: stderr >> |   _ | |
<UciProtocol (pid=8128)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=8128)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<Uc

--weights=lc0_windows\models\maia-1900.pb.gz
--weights=lc0_windows\models\maia-1400.pb.gz


<UciProtocol (pid=20108)>: stderr >> Loading weights file from: lc0_windows\models\maia-1400.pb.gz
<UciProtocol (pid=20108)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=20108)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=20108)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=20108)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=20108)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=20108)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=20108)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=20108)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=20108)>: stderr >> L2 cache capacity: 4194304
<UciProtocol (pid=8916)>: stderr >>        _
<UciProtocol (pid=8916)>: stderr >> |   _ | |
<UciProtocol (pid=8916)>: stderr >> |_ |_ |_| v0.31.1 built Aug 11 2024
<UciProtocol (pid=8916)>: stderr >> Found configuration file: c:\pc_coding\Finding-Elo\maia.chess\lc0_windows/lc0.config
<Uc

--weights=lc0_windows\models\maia-1900.pb.gz
--weights=lc0_windows\models\maia-1900.pb.gz


<UciProtocol (pid=3064)>: stderr >> Loading weights file from: lc0_windows\models\maia-1900.pb.gz
<UciProtocol (pid=3064)>: stderr >> Creating backend [cuda-auto]...
<UciProtocol (pid=3064)>: stderr >> Switching to [cuda-fp16]...
<UciProtocol (pid=3064)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=3064)>: stderr >> Latest version of CUDA supported by the driver: 12.6.0
<UciProtocol (pid=3064)>: stderr >> GPU: NVIDIA GeForce RTX 3070
<UciProtocol (pid=3064)>: stderr >> GPU memory: 7.99951 Gb
<UciProtocol (pid=3064)>: stderr >> GPU clock frequency: 1815 MHz
<UciProtocol (pid=3064)>: stderr >> GPU compute capability: 8.6
<UciProtocol (pid=3064)>: stderr >> L2 cache capacity: 4194304
